# (static) Biot equation

In this tutorial we present how to solve the (static) Biot equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$ and the pressure $p$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, $K$ the permeability of the porous medium and $s_0$ its storativity, and $\alpha$ the Biot-Willis coefficient fot the coupling between the two models.
We want to solve the following problem: find $({u}, p)$ such that
$$
    \begin{aligned}
         & \nabla \cdot [2 \mu {\epsilon}({u}) + \lambda \nabla \cdot u I
         - \alpha p {I}] = -{b}   \\
         & \partial_t (s_0 p + \alpha \nabla \cdot { u}) +
        \nabla \cdot \left( \mu^{-1} K \nabla p\right) = \psi
    \end{aligned}
    \quad \text{in } \Omega.
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. 

In particular, we consider only the spatial structure of the previous problem by doing only one time step. We call the latter $\Delta t$ and write the discrete problem as
$$
    \begin{bmatrix}
        K &  -\alpha D^\top \\
        \alpha D & s_0 M + \Delta t A
    \end{bmatrix}
    \begin{bmatrix}
        u \\ p
    \end{bmatrix}
    =
    \begin{bmatrix}
        b \\ \Delta t \psi
    \end{bmatrix}
$$
where $K$ is the stiffness matrix associated with the elastic problem, $D$ is the coupling between the two physics, $M$ is the mass matrix associated with the pressure and $A$ is the stiffness matrix associated to the diffusive part of the pressure. All the aforementioned matrices are properly scaled by their physical parameters if not explicitly written. The second row has been multiply by $\Delta t$ to preserve the skew-symmetry of the problem.

## Exercise 1: footing problem

A footing problem is when a force is impose on the top compressing the body and the bottom is fixed. In this case we assume that the force is applied on the central portion of the top boundary.

For this test case we set $\Omega = [0, 3] \times [0, 1]$, $b = 0$, and the following boundary conditions:
$$ 
\begin{aligned}
    &u = 0 &&\text{ and }&& \nu \cdot q = 0 &&\text{ on } \partial \Omega  \setminus \partial_{top} \Omega
\\
&\nu \cdot \sigma = 0 &&\text{ and }&& p = 0 &&\text{ on } \partial_{top} \Omega \setminus \partial_s \Omega
\\
&\nu \cdot \sigma = [0, -1]^\top &&\text{ and }&& p = 0 &&\text{ on }\partial_s \Omega 
\end{aligned}
$$
where $\partial_{s} \Omega = [1, 2] \times \{1\}$.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid. In this example we consider a 2-dimensional grid, but the presented code will work also in 3d.

In [2]:
mesh_size = 0.05
delta_t = 0.1
dim = 2

pts = np.array(
    [
        [0, 3, 3, 2, 1, 0],
        [0, 0, 1, 1, 1, 1],
    ]
)
sd = pg.grid_from_boundary_pts(pts, mesh_size, as_mdg=False)
sd.compute_geometry()

# NOTE: to be compliant with PorePy sparse matrices
sd.cell_faces = sps.csc_matrix(sd.cell_faces)

With the following code we set the data and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
# the physical parameters of the problem, assumed constant
alpha = 1
s0 = 1

E = 1
nu = 0.25
fun = -1e-3

lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))
mu = E / (2 * (1 + nu))

Let us set up the flow data.

In [4]:
key_p = "flow"

# Permeability
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))

# Boundary conditions
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]

# With the following steps we identify the portions of the boundary
# to impose the boundary conditions
top = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, b_faces], 1),
        sd.face_centers[0, b_faces] > 1,
        sd.face_centers[0, b_faces] < 2,
    )
)

# Set the labels for the boundary conditions
labels = np.array(["neu"] * b_faces.size)
labels[top] = "dir"

bc = pp.BoundaryCondition(sd, b_faces, labels)
bc_val = np.zeros(sd.num_faces)

# source term
source = np.zeros(sd.num_cells)

# collect all data
data_p = {
    pp.PARAMETERS: {
        key_p: {
            "second_order_tensor": perm,
            "bc_values": bc_val,
            "bc": bc,
            "source": source,
        }
    },
    pp.DISCRETIZATION_MATRICES: {key_p: {}},
}

Let us declare data associated to the elastic problem.

In [5]:
key_u = "mechanics"

# Create stiffness matrix
lambda_ = lambda_ * np.ones(sd.num_cells)
mu = mu * np.ones(sd.num_cells) / 2
C = pp.FourthOrderTensor(mu, lambda_)

# Define boundary type
b_faces = sd.get_all_boundary_faces()
num_b_faces = b_faces.size
labels = np.array(["neu"] * num_b_faces)

bottom = np.isclose(sd.face_centers[1, b_faces], 0)
labels[bottom] = "dir"
bc = pp.BoundaryConditionVectorial(sd, b_faces, labels)

bc_val = np.zeros((sd.dim, sd.num_faces))

top = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, :], 1),
        sd.face_centers[0, :] > 1,
        sd.face_centers[0, :] < 2,
    )
)
bc_val[1, top] = fun * sd.face_areas[top]
bc_val = bc_val.ravel("F")

# define source term
source = np.zeros((sd.dim, sd.num_cells)).ravel("F")

# collect all data
data_u = {
    pp.PARAMETERS: {
        key_u: {
            "fourth_order_tensor": C,
            "bc_values": bc_val,
            "bc": bc,
            "source": source,
            "scalar_vector_mappings": {key_u: 0.5},
        }
    },
    pp.DISCRETIZATION_MATRICES: {key_u: {}},
}

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [6]:
# discretization for the mechanical part
disc_u = pp.Mpsa(key_u)
disc_u.discretize(sd, data_u)

# discretization for the flow part
disc_p = pp.Mpfa(key_p)
disc_p.discretize(sd, data_p)

# discretization for the coupling part
disc_coupling = pp.Biot(key_u)
disc_coupling.discretize(sd, data_u)

dof_u, dof_p = sd.num_cells * sd.dim, sd.num_cells
dofs = np.array([dof_u, dof_p])

Let us create now the discrete problem.

In [7]:
# definition of the discretization matrices
K, rhs_u = disc_u.assemble_matrix_rhs(sd, data_u)
A, rhs_p = disc_p.assemble_matrix_rhs(sd, data_p)
M = sps.diags(sd.cell_volumes)

# the coupling term
D = data_u[pp.DISCRETIZATION_MATRICES][key_u]["displacement_divergence"][key_u]

# construction of the global problem
spp = sps.block_array(
    [
        [K, -D.T],
        [D, s0 * M + delta_t * A],
    ],
    format="csc",
)

rhs = np.hstack((rhs_u, rhs_p))

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [8]:
# NOTE: in case of the quasi static Biot model, the temporal for loop should be placed
# around the next lines of code

# solution of the problem
ls = pg.LinearSystem(spp, rhs)
x = ls.solve()

# split of the solution from the vector x
u, p = np.split(x, np.cumsum(dofs)[:-1])

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. Similarly for the flow field $q$. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [ ]:
# reshape the displacement for the export
u_3d = np.reshape(u, (sd.dim, -1), order="F")
if dim == 2:
    u_3d = np.vstack((u_3d, np.zeros(sd.num_cells)))

# export the final solution
save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu(
    [
        ("cell_p", p),
        ("cell_u", u_3d),
    ],
)

In [10]:
# Consistency check
assert np.isclose(np.linalg.norm(p), 0.011853312628645593)
assert np.isclose(np.linalg.norm(u), 0.02414002691914712)